In [442]:
import math

class Vector():

    def __init__(self, x=0, y=0, z=0):
        self._data = (x, y, z)
    
    def copy(self):
        return Vector(self)

    def __getitem__(self, n):
        return self._data[n]

    def __getattr__(self, name):
        if name == 'x':
            return self._data[0]
        elif name == 'y':
            return self._data[1]
        elif name == 'z':
            return self._data[2]

    def __setitem__(self, n, val):
        self._data[n] = val
        
    def __setattr__(self, name, val):
        if name == 'x':
            self._data[0] = val
        elif name == 'y':
            self._data[1] = val
        elif name == 'z':
            self._data[2] = val
        else:
            object.__setattr__(self, name, val)

    def __repr__(self):
        return '<Vector: ({:.4f}, {:.4f}, {:.4f})>'.format(*self._data)

    def __str__(self):
        return self.__repr__()

    def __eq__(self, other):
        diff = 0
        for i in range(3):
            diff = ( self._data[i] - other._data[i] ) ** 2
        return diff < 1e-06

    def __add__(self, other):
        a = self._data
        b = other._data
        return Vector(a[0] + b[0], a[1] + b[1], a[2] + b[2])
    
    def __rmul__(self, other):
        if type(other) in (int, float):
            a = self._data
            return Vector(other * a[0], other * a[1], other * a[2])
        elif type(other) == Vector:
            a = self._data
            b = other._data
            return a[0] * b[0] + a[1] * b[1] + a[2] * b[2]
        
    __mul__ = __rmul__ # to support b*2 == 2*b

    def __neg__(self):
        return -1 * self

    def __sub__(self, other):
        return self + -1 * other
    
    def __truediv__(self, other):
        return (1.0 / other) * self  
    
    __div__ = __truediv__

    def __abs__(self):
        return math.sqrt(self * self)
    
    def vmag(self):
        return abs(self)
    
    def vdir(self):
        return self / abs(self)
    
    def vect(mag, ang, deg=True):
        if deg: # then convert to radians
            ang = math.radians(ang)
        return mag * Vector(math.cos(ang), 0, math.sin(ang))

    def vang(self, other=None, deg=True):
        if other is None:
            other = self - self.proj(Vector(0,0,1))
        cos_theta = self.vdir() * other.vdir()
        vang = math.acos(cos_theta)
        if deg:
            vang = math.degrees(vang)
        return vang
        
    def proj(self, other, normalize=True):
        if normalize:
            v_hat = other.vdir()
        else:
            v_hat = other
        return (self * v_hat) * v_hat

Vector.ex = Vector.e0 = Vector(1, 0, 0)
Vector.ey = Vector.e1 = Vector(0, 1, 0)
Vector.ez = Vector.e2 = Vector(0, 0, 1)

In [443]:
a = Vector(1,1,1)

In [444]:
a.vmag()

1.7320508075688772

In [451]:
x = Vector.ex
x

<Vector: (1.0000, 0.0000, 0.0000)>

In [456]:
3*a.vdir().proj(x) + Vector.ey

<Vector: (1.7321, 1.0000, 0.0000)>